In [1]:
import pandas as pd
from sklearn.utils import resample
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import pickle

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix

In [2]:
df = pd.read_csv("loan_detection.csv")

In [3]:
df_cpy = df.copy()

In [4]:
df.head()

,age,campaign,pdays,previous,no_previous_contact,not_working,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,...,month_sep,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success,Loan_Status_label
0,56,1,999,0,1,0,0,0,0,1,...,0,0,1,0,0,0,0,1,0,0
1,57,1,999,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
2,37,1,999,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
3,40,1,999,0,1,0,1,0,0,0,...,0,0,1,0,0,0,0,1,0,0
4,56,1,999,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0


In [5]:
df.columns

Index(['age', 'campaign', 'pdays', 'previous', 'no_previous_contact',
       'not_working', 'job_admin.', 'job_blue-collar', 'job_entrepreneur',
       'job_housemaid', 'job_management', 'job_retired', 'job_self-employed',
       'job_services', 'job_student', 'job_technician', 'job_unemployed',
       'job_unknown', 'marital_divorced', 'marital_married', 'marital_single',
       'marital_unknown', 'education_basic.4y', 'education_basic.6y',
       'education_basic.9y', 'education_high.school', 'education_illiterate',
       'education_professional.course', 'education_university.degree',
       'education_unknown', 'default_no', 'default_unknown', 'default_yes',
       'housing_no', 'housing_unknown', 'housing_yes', 'loan_no',
       'loan_unknown', 'loan_yes', 'contact_cellular', 'contact_telephone',
       'month_apr', 'month_aug', 'month_dec', 'month_jul', 'month_jun',
       'month_mar', 'month_may', 'month_nov', 'month_oct', 'month_sep',
       'day_of_week_fri', 'day_of_week_mon'

In [6]:
df['previous'].unique()

array([0, 1, 2, 3, 4, 5, 6, 7])

In [7]:
df['no_previous_contact'].unique()

array([1, 0])

In [8]:
df = df.drop("no_previous_contact", axis=1)


In [9]:
df.columns

Index(['age', 'campaign', 'pdays', 'previous', 'not_working', 'job_admin.',
       'job_blue-collar', 'job_entrepreneur', 'job_housemaid',
       'job_management', 'job_retired', 'job_self-employed', 'job_services',
       'job_student', 'job_technician', 'job_unemployed', 'job_unknown',
       'marital_divorced', 'marital_married', 'marital_single',
       'marital_unknown', 'education_basic.4y', 'education_basic.6y',
       'education_basic.9y', 'education_high.school', 'education_illiterate',
       'education_professional.course', 'education_university.degree',
       'education_unknown', 'default_no', 'default_unknown', 'default_yes',
       'housing_no', 'housing_unknown', 'housing_yes', 'loan_no',
       'loan_unknown', 'loan_yes', 'contact_cellular', 'contact_telephone',
       'month_apr', 'month_aug', 'month_dec', 'month_jul', 'month_jun',
       'month_mar', 'month_may', 'month_nov', 'month_oct', 'month_sep',
       'day_of_week_fri', 'day_of_week_mon', 'day_of_week_thu',
  

In [10]:
for i in df.columns:
    print(i)
    print(df[i].unique())
    print()

age
[56 57 37 40 45 59 41 24 25 29 35 54 46 50 39 30 55 49 34 52 58 32 38 44
 42 60 53 47 51 48 33 31 43 36 28 27 26 22 23 20 21 61 19 18 70 66 76 67
 73 88 95 77 68 75 63 80 62 65 72 82 64 71 69 78 85 79 83 81 74 17 87 91
 86 98 94 84 92 89]

campaign
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 19 18 23 14 22 25 16 17 15 20 56
 39 35 42 28 26 27 32 21 24 29 31 30 41 37 40 33 34 43]

pdays
[999   6   4   3   5   1   0  10   7   8   9  11   2  12  13  14  15  16
  21  17  18  22  25  26  19  27  20]

previous
[0 1 2 3 4 5 6 7]

not_working
[0 1]

job_admin.
[0 1]

job_blue-collar
[0 1]

job_entrepreneur
[0 1]

job_housemaid
[1 0]

job_management
[0 1]

job_retired
[0 1]

job_self-employed
[0 1]

job_services
[0 1]

job_student
[0 1]

job_technician
[0 1]

job_unemployed
[0 1]

job_unknown
[0 1]

marital_divorced
[0 1]

marital_married
[1 0]

marital_single
[0 1]

marital_unknown
[0 1]

education_basic.4y
[1 0]

education_basic.6y
[0 1]

education_basic.9y
[0 1]

education_high.school
[0 1]


In [11]:
job_cols = [
    'not_working',
    'job_admin.',
    'job_blue-collar',
    'job_entrepreneur',
    'job_housemaid',
    'job_management',
    'job_retired',
    'job_self-employed',
    'job_services',
    'job_student',
    'job_technician',
    'job_unemployed',
    'job_unknown'
]
df["job"] = df[job_cols].idxmax(axis=1)
df["job"] = df["job"].str.replace("job_", "", regex=False)
df["job"] = df["job"].replace("unknown","other")

df.drop(columns=job_cols, inplace=True)


In [12]:
df.columns

Index(['age', 'campaign', 'pdays', 'previous', 'marital_divorced',
       'marital_married', 'marital_single', 'marital_unknown',
       'education_basic.4y', 'education_basic.6y', 'education_basic.9y',
       'education_high.school', 'education_illiterate',
       'education_professional.course', 'education_university.degree',
       'education_unknown', 'default_no', 'default_unknown', 'default_yes',
       'housing_no', 'housing_unknown', 'housing_yes', 'loan_no',
       'loan_unknown', 'loan_yes', 'contact_cellular', 'contact_telephone',
       'month_apr', 'month_aug', 'month_dec', 'month_jul', 'month_jun',
       'month_mar', 'month_may', 'month_nov', 'month_oct', 'month_sep',
       'day_of_week_fri', 'day_of_week_mon', 'day_of_week_thu',
       'day_of_week_tue', 'day_of_week_wed', 'poutcome_failure',
       'poutcome_nonexistent', 'poutcome_success', 'Loan_Status_label', 'job'],
      dtype='object')

In [13]:
mar_status = ['marital_divorced','marital_married', 'marital_single', 'marital_unknown']

df["marital_status"] = df[mar_status].idxmax(axis=1)
df["marital_status"] = df["marital_status"].str.replace("marital_", "", regex=False)
df["marital_status"] = df["marital_status"].replace("unknown","other")

df.drop(columns=mar_status, inplace=True)


In [14]:
edu_status = [ 'education_basic.4y', 'education_basic.6y', 'education_basic.9y',
       'education_high.school', 'education_illiterate',
       'education_professional.course', 'education_university.degree',
       'education_unknown']

df["education"] = df[edu_status].idxmax(axis=1)
df["education"] = df["education"].str.replace("education_", "", regex=False)
df["education"] = df["education"].replace("unknown","other")

df.drop(columns=edu_status, inplace=True)


In [15]:
def_status = ['default_no', 'default_unknown', 'default_yes']

df["default"] = df[def_status].idxmax(axis=1)
df["default"] = df["default"].str.replace("default_", "", regex=False)
df["default"] = df["default"].replace("unknown","other")
 
df.drop(columns=def_status, inplace=True)

In [16]:
house_status = ['housing_no', 'housing_unknown', 'housing_yes']

df["house"] = df[house_status].idxmax(axis=1)
df["house"] = df["house"].str.replace("housing_", "", regex=False)
df["house"] = df["house"].replace("unknown","other")
 
df.drop(columns=house_status, inplace=True)

In [17]:
loan_status = ['loan_no','loan_unknown', 'loan_yes']

df["loan"] = df[loan_status].idxmax(axis=1)
df["loan"] = df["loan"].str.replace("loan_", "", regex=False)
df["loan"] = df["loan"].replace("unknown","other")
 
df.drop(columns=loan_status, inplace=True)

In [18]:
df = df.drop(['contact_cellular', 'contact_telephone'], axis = 1)

In [19]:
df.columns

Index(['age', 'campaign', 'pdays', 'previous', 'month_apr', 'month_aug',
       'month_dec', 'month_jul', 'month_jun', 'month_mar', 'month_may',
       'month_nov', 'month_oct', 'month_sep', 'day_of_week_fri',
       'day_of_week_mon', 'day_of_week_thu', 'day_of_week_tue',
       'day_of_week_wed', 'poutcome_failure', 'poutcome_nonexistent',
       'poutcome_success', 'Loan_Status_label', 'job', 'marital_status',
       'education', 'default', 'house', 'loan'],
      dtype='object')

In [20]:
month = ['month_apr', 'month_aug',
       'month_dec', 'month_jul', 'month_jun', 'month_mar', 'month_may',
       'month_nov', 'month_oct', 'month_sep']

df["month"] = df[month].idxmax(axis=1)
df["month"] = df["month"].str.replace("month_", "", regex=False)
 
df.drop(columns=month, inplace=True)

In [21]:
week = ['day_of_week_fri',
       'day_of_week_mon', 'day_of_week_thu', 'day_of_week_tue',
       'day_of_week_wed']

df["week"] = df[week].idxmax(axis=1)
df["week"] = df["week"].str.replace("day_of_week_", "", regex=False)
 
df.drop(columns=week, inplace=True)

In [22]:
poutcome = ['poutcome_failure', 'poutcome_nonexistent',
       'poutcome_success']

df["poutcome"] = df[poutcome].idxmax(axis=1)
df["poutcome"] = df["poutcome"].str.replace("poutcome_", "", regex=False)
 
df.drop(columns=poutcome, inplace=True)

In [23]:
df.columns

Index(['age', 'campaign', 'pdays', 'previous', 'Loan_Status_label', 'job',
       'marital_status', 'education', 'default', 'house', 'loan', 'month',
       'week', 'poutcome'],
      dtype='object')

In [24]:
df.head()

,age,campaign,pdays,previous,Loan_Status_label,job,marital_status,education,default,house,loan,month,week,poutcome
0,56,1,999,0,0,housemaid,married,basic.4y,no,no,no,may,mon,nonexistent
1,57,1,999,0,0,services,married,high.school,other,no,no,may,mon,nonexistent
2,37,1,999,0,0,services,married,high.school,no,yes,no,may,mon,nonexistent
3,40,1,999,0,0,admin.,married,basic.6y,no,no,no,may,mon,nonexistent
4,56,1,999,0,0,services,married,high.school,no,no,yes,may,mon,nonexistent


In [25]:
col = "Loan_Status_label"

df = df[[c for c in df.columns if c != col] + [col]]


In [26]:
df.head()

,age,campaign,pdays,previous,job,marital_status,education,default,house,loan,month,week,poutcome,Loan_Status_label
0,56,1,999,0,housemaid,married,basic.4y,no,no,no,may,mon,nonexistent,0
1,57,1,999,0,services,married,high.school,other,no,no,may,mon,nonexistent,0
2,37,1,999,0,services,married,high.school,no,yes,no,may,mon,nonexistent,0
3,40,1,999,0,admin.,married,basic.6y,no,no,no,may,mon,nonexistent,0
4,56,1,999,0,services,married,high.school,no,no,yes,may,mon,nonexistent,0


In [27]:
for i in df.columns:
    print(i)
    print(df[i].value_counts())
    print()

age
age
31    1947
32    1846
33    1833
36    1780
35    1759
      ... 
91       2
98       2
95       1
87       1
94       1
Name: count, Length: 78, dtype: int64

campaign
campaign
1     17642
2     10570
3      5341
4      2651
5      1599
6       979
7       629
8       400
9       283
10      225
11      177
12      125
13       92
14       69
17       58
16       51
15       51
18       33
20       30
19       26
21       24
22       17
23       16
24       15
27       11
29       10
28        8
26        8
25        8
30        7
31        7
35        5
33        4
32        4
34        3
43        2
40        2
42        2
39        1
56        1
37        1
41        1
Name: count, dtype: int64

pdays
pdays
999    39673
3        439
6        412
4        118
9         64
2         61
7         60
12        58
10        52
5         46
13        36
11        28
1         26
15        24
14        20
8         18
0         15
16        11
17         8
18         7
22         

In [28]:
df_majority = df[df['Loan_Status_label'] == 0]
df_minority = df[df['Loan_Status_label'] == 1]

In [29]:
df_minority_upsampled = resample(df_minority, replace = True, n_samples = len(df_majority), random_state = 42)

In [30]:
df = pd.concat([df, df_minority_upsampled], axis=0) 

In [31]:
df['Loan_Status_label'].value_counts()

Loan_Status_label
1    41188
0    36548
Name: count, dtype: int64

In [32]:
df['education'].unique()

array(['basic.4y', 'high.school', 'basic.6y', 'basic.9y',
       'professional.course', 'other', 'university.degree', 'illiterate'],
      dtype=object)

In [33]:
df['education'] = df['education'].map({'basic.4y' : '4 year', 'high.school' :'high_school', 'basic.6y': '6 year', 'basic.9y' : '9 year',
       'professional.course' : 'professional_course', 'other' : 'other', 'university.degree' : 'university_degree', 'illiterate' : 'illiterate'})

In [34]:
df['education'].unique()

array(['4 year', 'high_school', '6 year', '9 year', 'professional_course',
       'other', 'university_degree', 'illiterate'], dtype=object)

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 77736 entries, 0 to 20426
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   age                77736 non-null  int64 
 1   campaign           77736 non-null  int64 
 2   pdays              77736 non-null  int64 
 3   previous           77736 non-null  int64 
 4   job                77736 non-null  object
 5   marital_status     77736 non-null  object
 6   education          77736 non-null  object
 7   default            77736 non-null  object
 8   house              77736 non-null  object
 9   loan               77736 non-null  object
 10  month              77736 non-null  object
 11  week               77736 non-null  object
 12  poutcome           77736 non-null  object
 13  Loan_Status_label  77736 non-null  int64 
dtypes: int64(5), object(9)
memory usage: 8.9+ MB


In [36]:
df['job'] = df['job'].str.replace("-", '_')
df['job'] = df['job'].str.replace(".", '')

In [37]:
for i in df.columns:
    print(i)
    print(df[i].unique())
    print()

age
[56 57 37 40 45 59 41 24 25 29 35 54 46 50 39 30 55 49 34 52 58 32 38 44
 42 60 53 47 51 48 33 31 43 36 28 27 26 22 23 20 21 61 19 18 70 66 76 67
 73 88 95 77 68 75 63 80 62 65 72 82 64 71 69 78 85 79 83 81 74 17 87 91
 86 98 94 84 92 89]

campaign
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 19 18 23 14 22 25 16 17 15 20 56
 39 35 42 28 26 27 32 21 24 29 31 30 41 37 40 33 34 43]

pdays
[999   6   4   3   5   1   0  10   7   8   9  11   2  12  13  14  15  16
  21  17  18  22  25  26  19  27  20]

previous
[0 1 2 3 4 5 6 7]

job
['housemaid' 'services' 'admin' 'blue_collar' 'technician' 'not_working'
 'management' 'self_employed' 'other' 'entrepreneur']

marital_status
['married' 'single' 'divorced' 'other']

education
['4 year' 'high_school' '6 year' '9 year' 'professional_course' 'other'
 'university_degree' 'illiterate']

default
['no' 'other' 'yes']

house
['no' 'yes' 'other']

loan
['no' 'yes' 'other']

month
['may' 'jun' 'jul' 'aug' 'oct' 'nov' 'dec' 'mar' 'apr' 'sep']

week
['mon'

In [38]:

job_map = {
    'housemaid':0,
    'services':1,
    'admin':2,
    'blue_collar':3,
    'technician':4,
    'not_working':5,
    'management':6,
    'self_employed':7,
    'entrepreneur':8,
    'other':9
}

marital_map = {
    'married':0,
    'single':1,
    'divorced':2,
    'other':3
}

education_map = {
    'illiterate':0,
    '4 year':1,
    '6 year':2,
    '9 year':3,
    'high_school':4,
    'professional_course':5,
    'university_degree':6,
    'other':7
}
month_map = {
    "jan":1, "feb":2, "mar":3, "apr":4,
    "may":5, "jun":6, "jul":7, "aug":8,
    "sep":9, "oct":10, "nov":11, "dec":12
}
week_map = {
    "mon":1,
    "tue":2,
    "wed":3,
    "thu":4,
    "fri":5,
    "sat" : 6,
    "sun" : 7
}



In [39]:
df["job"] = df["job"].map(job_map)
df["marital_status"] = df["marital_status"].map(marital_map)
df["education"] = df["education"].map(education_map)
df["month"] = df["month"].map(month_map)
df["week"] = df["week"].map(week_map)

In [40]:
bundle = {
    "job_map": job_map,
    "marital_map": marital_map,
    "education_map": education_map,
    "month_map" : month_map,
    "week_map" : week_map
}

pickle.dump(bundle, open("encoder_bundle.pkl","wb"))

In [41]:
ohe_cols = ["default","house","loan","poutcome"]
ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
encoded = ohe.fit_transform(df[ohe_cols])
encoded_df = pd.DataFrame(
    encoded,
    columns=ohe.get_feature_names_out(ohe_cols),
    index=df.index
)

# replace original columns
df_final = df.drop(columns=ohe_cols)
df = pd.concat([df_final, encoded_df], axis=1)

In [42]:
bundle2 = {
    "encoder": ohe,
    "ohe_cols": ohe_cols,
    "feature_order": df_final.columns.tolist()
}

with open("ohe.pkl","wb") as f:
    pickle.dump(bundle2,f)


In [43]:
df.head(5)

,age,campaign,pdays,previous,job,marital_status,education,month,week,Loan_Status_label,...,default_yes,house_no,house_other,house_yes,loan_no,loan_other,loan_yes,poutcome_failure,poutcome_nonexistent,poutcome_success
0,56,1,999,0,0,0,1,5,1,0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,57,1,999,0,1,0,4,5,1,0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,37,1,999,0,1,0,4,5,1,0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
3,40,1,999,0,2,0,2,5,1,0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,56,1,999,0,1,0,4,5,1,0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [44]:
df.columns

Index(['age', 'campaign', 'pdays', 'previous', 'job', 'marital_status',
       'education', 'month', 'week', 'Loan_Status_label', 'default_no',
       'default_other', 'default_yes', 'house_no', 'house_other', 'house_yes',
       'loan_no', 'loan_other', 'loan_yes', 'poutcome_failure',
       'poutcome_nonexistent', 'poutcome_success'],
      dtype='object')

In [45]:
df.head()

,age,campaign,pdays,previous,job,marital_status,education,month,week,Loan_Status_label,...,default_yes,house_no,house_other,house_yes,loan_no,loan_other,loan_yes,poutcome_failure,poutcome_nonexistent,poutcome_success
0,56,1,999,0,0,0,1,5,1,0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,57,1,999,0,1,0,4,5,1,0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,37,1,999,0,1,0,4,5,1,0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
3,40,1,999,0,2,0,2,5,1,0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,56,1,999,0,1,0,4,5,1,0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [46]:
df.columns

Index(['age', 'campaign', 'pdays', 'previous', 'job', 'marital_status',
       'education', 'month', 'week', 'Loan_Status_label', 'default_no',
       'default_other', 'default_yes', 'house_no', 'house_other', 'house_yes',
       'loan_no', 'loan_other', 'loan_yes', 'poutcome_failure',
       'poutcome_nonexistent', 'poutcome_success'],
      dtype='object')

In [47]:
X = df.iloc[:,:-1]
Y = df.iloc[:,-1]

In [48]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)

In [49]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [50]:
model = LogisticRegression()

In [51]:
model.fit(X_train_scaled, Y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [52]:
model.score(X_train_scaled, Y_train)

1.0

In [53]:
model.score(X_test_scaled, Y_test)

1.0

In [54]:
y_pred = model.predict(X_test_scaled)

In [55]:
confusion_matrix(Y_test, y_pred)

array([[13836,     0],
       [    0,  1712]])

In [56]:
pickle.dump(model, open("model.pkl","wb"))